# Assignment 3

In [16]:
# importing necessary packages
import pandas as pd
import tarfile

## Problem 1: Spam and Ham data

In [17]:
def read_data(fn):
    dataset = list()
    with tarfile.open(fn) as tf:
        for i in tf:
            if i.isfile():
                with tf.extractfile(i) as f:
                    b  = f.read()
                    if b'charset=utf-8' in b or b'charset=UTF-8' in b:
                        s = b.decode('utf-8')
                    else:
                        s = b.decode('iso-8859-1')
                    dataset.append(s)
    return dataset

In [18]:
data = {
    'easy_ham': read_data('data/20021010_easy_ham.tar.bz2'),
    'hard_ham': read_data('data/20021010_hard_ham.tar.bz2'),
    'spam': read_data('data/20021010_spam.tar.bz2')
}

In [19]:
df_eh = pd.DataFrame(data['easy_ham']) 
df_hh = pd.DataFrame(data['hard_ham'])
df_spam = pd.DataFrame(data['spam'])

In [20]:
df_eh.rename(columns={0: 'email'}, inplace=True)
df_hh.rename(columns={0: 'email'}, inplace=True)
df_spam.rename(columns={0: 'email'}, inplace=True)

df_eh['label'] = 'easy_ham'
df_hh['label'] = 'hard_ham'
df_spam['label'] = 'spam'

In [21]:
df_spam

,email,label
0,From george@vccomputers.ie Mon Aug 26 17:49:4...,spam
1,From gwfqjulie@msn.com Mon Aug 26 21:37:20 20...,spam
2,From simply-amateur-zzzz=example.com@free4porn...,spam
3,From thisisagreatfreepornmovie@framesetup.com ...,spam
4,From safety33o@l4.newnamedns.com Tue Aug 27 0...,spam
...,...,...
496,From havoc1006@yahoo.com Mon Aug 26 15:49:43 ...,spam
497,From mando@insiq.us Mon Aug 26 15:49:52 2002\...,spam
498,From girl_with_toys_541652k57@yahoo.com Mon A...,spam
499,From guyhaibo@yahoo.ca Mon Aug 26 15:50:05 20...,spam


In [22]:
df_eh.to_csv("eh_content.csv")
df_hh.to_csv("hh_content.csv")
df_spam.to_csv("spam_content.csv")

B. Data Splitting

In [23]:
from sklearn.model_selection import train_test_split

# Train-test split for df_spam
X_train_spam, X_test_spam, y_train_spam, y_test_spam = train_test_split(df_spam['email'], df_spam['label'], test_size=0.2, random_state=42)

# Train-test split for df_eh
X_train_eh, X_test_eh, y_train_eh, y_test_eh = train_test_split(df_eh['email'], df_eh['label'], test_size=0.2, random_state=42)

# Train-test split for df_hh
X_train_hh, X_test_hh, y_train_hh, y_test_hh = train_test_split(df_hh['email'], df_hh['label'], test_size=0.2, random_state=42)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()

X_train_spam_counts = count.fit_transform(X_train_spam)
X_train_eh_counts = count.fit_transform(X_train_eh)
X_train_hh_counts = count.fit_transform(X_train_hh)

# Apply the same transformation on the test data
X_test_spam_counts = count.transform(X_test_spam)
X_test_eh_counts = count.transform(X_test_eh)
X_test_hh_counts = count.transform(X_test_hh)

Problem 3

In [28]:
# Merge the training data for spam and ham
X_train = pd.concat([X_train_spam_counts, X_train_eh_counts],ignore_index=True)
y_train = pd.concat([y_train_spam, y_train_eh], ignore_index=True)

TypeError: cannot concatenate object of type '<class 'scipy.sparse._csr.csr_matrix'>'; only Series and DataFrame objs are valid

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Instantiate classifiers
multinomial_nb = MultinomialNB()
bernoulli_nb = BernoulliNB()

In [ ]:
# Train the classifiers on the training data
multinomial_nb.fit(X_train_eh_counts, y_train_eh)
bernoulli_nb.fit(X_train_eh_counts, y_train_eh)

# Make predictions on the test set
multinomial_nb_preds = multinomial_nb.predict(X_test_eh)
bernoulli_nb_preds = bernoulli_nb.predict(X_test_eh)

